AS2016465

K.L.H.S.Perera

In [0]:
g_match,g_mismatch,g_gap= 1,-1,-1
alignment_list_needle ,alignment_list_waterman = [],[]
g_seq1, g_seq2, g_max_cells = [], [], []
pointer_matrix_needle =  [[0 for i in range(len(g_seq2) + 1)] for j in range(len(g_seq1) + 1)]
pointer_matrix_waterman =  [[0 for i in range(len(g_seq2) + 1)] for j in range(len(g_seq1) + 1)]

#Function will be used to initialize the scoring scheme
def scoring(match, mismatch, gap):
    global g_match,g_mismatch,g_gap
    g_match = match
    g_mismatch = mismatch
    g_gap = gap

#Function to determine score between any two bases in alignment
def match_score(b1, b2):
    if b1 == b2:
        return g_match
    else:
        return g_mismatch

#Function return values for pointer matrix
def pointers(di,le,up):
    pointer = max(di,le,up) 
    if pointer == di and pointer == le and pointer == up:
      return 'D-L-U'
    elif  pointer == di and pointer == le: 
      return 'D-L'
    elif  pointer == di and pointer == up: 
      return 'D-U' 
    elif  pointer == up and pointer == le: 
      return 'U-L'    
    elif di == pointer:
        return 'D'
    elif le == pointer:
        return 'L'
    else:
        return 'U'

#Function to find place of cell with highest score(waterman)
def find_max_score(score_matrix,n,m):
  global g_max_cells
  max_score = score_matrix[0][0]
  for i in range(1,m):
        for j in range(1,n):
            if(max_score < score_matrix[i][j]):
               max_score = score_matrix[i][j]
  for i in range(1,m):
        for j in range(1,n):
            if(max_score == score_matrix[i][j] and max_score != 0 ):
               g_max_cells.append(i)
               g_max_cells.append(j) 

# This method calculate the all cell values of the score matrix and pointer matrix
def calculateScore(n,m):
  global pointer_matrix_needle 
  global pointer_matrix_waterman 
  score_matrix_waterman = [[0 for x in range(n)] for y in range(m)] 
  pointer_matrix_waterman = [[0 for x in range(n)] for y in range(m)] 
  score_matrix_needle = [[0 for x in range(n)] for y in range(m)] 
  pointer_matrix_needle = [[0 for x in range(n)] for y in range(m)] 
    
  for i in range(m):   #Fill first row and column in the matrix
        score_matrix_needle[i][0] = g_gap * i
        pointer_matrix_needle[i][0] = 'U'
        score_matrix_waterman[i][0] = 0
        pointer_matrix_waterman[i][0] = 'S'
  for j in range(n):
        score_matrix_needle[0][j] = g_gap * j 
        pointer_matrix_needle[0][j] = 'L'
        score_matrix_waterman[0][j] = 0
        pointer_matrix_waterman[0][j] = 'S'
  
  for i in range(1,m):   #Fill score_matrix with the correct values
        for j in range(1,n):
            # score_matrix_needle
            left = score_matrix_needle[i][j-1] + g_gap  #Left cell value
            up = score_matrix_needle[i-1][j] + g_gap  #Upper cell value
            diagonal = score_matrix_needle[i-1][j-1] + match_score(g_seq1[j-1], g_seq2[i-1])  #Diagonal cell value
            score_matrix_needle[i][j] = max(diagonal,left,up) 
            pointer_matrix_needle[i][j] = pointers(diagonal,left,up)
            # score_matrix_waterman
            left = score_matrix_waterman[i][j-1] + g_gap  #Left cell value
            up = score_matrix_waterman[i-1][j] + g_gap  #Upper cell value
            diagonal = score_matrix_waterman[i-1][j-1] + match_score(g_seq1[j-1], g_seq2[i-1])  #Diagonal cell value
            cell_value = max(diagonal,left,up)
            if (cell_value <= 0):
              score_matrix_waterman[i][j] = 0 
              pointer_matrix_waterman[i][j] = 'S'
            else:
              score_matrix_waterman[i][j] = cell_value  
              pointer_matrix_waterman[i][j] = pointers(diagonal,left,up)
  return score_matrix_waterman            
  
#Function to perform traceback to (0,0) 
def traceback(i, j, align_1, align_2):
  if (len(g_max_cells) != 0):   
    if(pointer_matrix_waterman[i][j].find('S') != -1):
       global alignment_list_waterman
       alignment_list_waterman.append(align_1 [::-1])
       alignment_list_waterman.append(align_2 [::-1])
       return
  else:    
    if (i == 0 and j == 0):
      global alignment_list_needle
      alignment_list_needle.append(align_1 [::-1])
      alignment_list_needle.append(align_2 [::-1])
      return
  
  if ((pointer_matrix_needle[i][j].find('U') != -1 and (len(g_max_cells) == 0)) or (pointer_matrix_waterman[i][j].find('U') != -1 and (len(g_max_cells) != 0))):
    traceback(i -1, j, align_1 + '-', align_2 + g_seq2[i-1])
  if ((pointer_matrix_needle[i][j].find('D') != -1 and (len(g_max_cells) == 0)) or (pointer_matrix_waterman[i][j].find('D') != -1 and (len(g_max_cells) != 0))):
    traceback(i -1, j - 1, align_1 + g_seq2[i - 1], align_2 + g_seq1[j - 1])
  if ((pointer_matrix_needle[i][j].find('L') != -1 and (len(g_max_cells) == 0)) or (pointer_matrix_waterman[i][j].find('L') != -1) and (len(g_max_cells) != 0)):
    traceback(i, j - 1, align_1 + g_seq1[j-1], align_2 + '-')

#Function to perform needleman-wunsch algorithm
def needle(seq_1,seq_2):
  global g_seq1, g_seq2
  g_seq1 = list(seq_1)
  g_seq2 = list(seq_2)
  calculateScore(len(seq_1)+1,len(seq_2)+1)
  traceback(len(g_seq2),len(g_seq1),'','')

#Function to perform smith-waterman algorithm
def waterman(seq_1, seq_2):
  global g_seq1, g_seq2
  g_seq1 = list(seq_1)
  g_seq2 = list(seq_2)
  score_matrix = calculateScore(len(seq_1)+1,len(seq_2)+1)
  find_max_score(score_matrix,len(seq_1)+1,len(seq_2)+1)
  i=0
  j=0
  while i < len(g_max_cells)/2:
    traceback(g_max_cells[j],g_max_cells[j+1],'','')
    i+=1
    j+=2

#Function to get the user inputs
def inputs():
  global g_seq1, g_seq2, g_match, g_mismatch, g_gap
  g_seq1 = input("Enter sequence 1:")
  g_seq2 = input("Enter sequence 2:")
  yes = input("Do you want to change score?\n(Enter Y for yes)")
  if (yes == 'Y'):
    g_match = int(input("Enter match score:"))
    g_mismatch = int(input("Enter mismatch score:"))
    g_gap = int(input("Enter gap score:"))
    scoring(g_match,g_mismatch,g_gap)

#Fuction to print the results
def print_results(g_seq1,g_seq2):
  i = 0
  print("Sequences:")
  str1,str2 = "",""
  print(str1.join(g_seq1),"and",str2.join(g_seq2)) 
  print("******************************************************************")
  print("Needleman-Wunsch algorithm for Global Alignment of two sequences")         
  i = 0
  while i < len(alignment_list_needle):
    print("Alignment", (i + 2 * 1)//2, ":", alignment_list_needle[i], "and", alignment_list_needle[i + 1])
    i += 2
  print("******************************************************************")
  print("Smith-Waterman algorithm for Local Alignment of two sequences")   
  i = 0
  while i < len(alignment_list_waterman):
    print("Alignment", (i + 2 * 1)//2, ":", alignment_list_waterman[i], "and", alignment_list_waterman[i + 1])
    i += 2
inputs()
needle(g_seq1,g_seq2)
waterman(g_seq1,g_seq2)
print_results(g_seq1,g_seq2)


Enter sequence 1:ATTAC
Enter sequence 2:AATTC
Do you want to change score?
(Enter Y for yes)A
Sequences:
ATTAC and AATTC
******************************************************************
Needleman-Wunsch algorithm for Global Alignment of two sequences
Alignment 1 : A-TTAC and AATT-C
Alignment 2 : -ATTAC and AATT-C
******************************************************************
Smith-Waterman algorithm for Local Alignment of two sequences
Alignment 1 : ATT and ATT
Alignment 2 : ATTAC and ATT-C
